# Magic Facies Prediction

In [28]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import sys
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from plotly import graph_objs as go
from plotly import tools
# Import our custom modules
sys.path.append("..")
from mysticbit import ml, munging, plots

## Data Wrangling

In [66]:
df = munging.load_log_data()
df.head()

,PSEUDO_DEPTH,TEMP,CALI,GR,ILD,NPHI,RHOB,DT,TVD,TVDSS,...,PHIE,PHIT,SW,EF,PERFOS,RES_ID,WELL_ID,X,Y,HACKANAME
0,2074.53,87.19,10.31,55.08,3.3128,0.1678,2.3124,108.95,1588.62,1566.62,...,0.0591,0.2066,0.5863,5,0,3005,210075859,15,20.0,B03
1,2074.68,87.20,10.25,52.89,3.6592,0.1739,2.2895,110.91,1588.77,1566.77,...,0.0871,0.2222,0.5225,5,0,3005,210075859,15,20.0,B03
2,2074.83,87.20,10.20,52.54,4.1220,0.1618,2.2686,114.92,1588.92,1566.92,...,0.0730,0.2061,0.5004,5,0,3005,210075859,15,20.0,B03
3,2074.98,87.21,10.18,54.36,4.7189,0.1448,2.2533,119.66,1589.06,1567.06,...,0.0335,0.1768,0.4566,5,0,3005,210075859,15,20.0,B03
4,2075.90,87.24,9.76,48.13,13.0483,0.1085,2.0885,126.26,1589.94,1567.94,...,0.0331,0.1412,0.2211,1,1,3005,210075859,15,20.0,B03


In [67]:
print(f"Titles: {' '.join(list(df))}")

Titles: PSEUDO_DEPTH TEMP CALI GR ILD NPHI RHOB DT TVD TVDSS VCL PHIE PHIT SW EF PERFOS RES_ID WELL_ID X Y HACKANAME


## Facies Data Generation

As we don't have any facies data to form our training dataset we need to generate some from the well logs we do have:
- GR - Gamma Ray
- RHOB - Density
- NPHI - Neutron Density
- DT - Sonic Log

Then split it into facies using K-Means clustering

In [72]:
df['FACIES'] = ml.create_facies(df, 5)

In [73]:
log = df[df['HACKANAME'] == 'B03']

In [74]:
log = log[['TVDSS', 'GR', 'RHOB', 'NPHI', 'DT', 'FACIES']]

In [75]:
gamma_ray = go.Scatter(
    x = log['GR'],
    y = log['TVDSS'],
    name = 'GR'
)

density = go.Scatter(
    x = log['RHOB'],
    y = log['TVDSS'],
    name = 'RHOB'
)

neutron_density = go.Scatter(
    x = log['NPHI'],
    y = log['TVDSS'],
    name = 'NPHI'
)

sonic_log = go.Scatter(
    x = log['DT'],
    y = log['TVDSS'],
    name = 'DT'
)

facies_estimation = go.Scatter(
    x = log['FACIES'],
    y = log['TVDSS'],
    name = 'Facies'
)

layout = go.Layout(
    autosize=False,
    width=200,
    height=1000
)

fig = tools.make_subplots(rows=1, cols=5)

fig.append_trace(gamma_ray, 1, 1)
fig.append_trace(density, 1, 2)
fig.append_trace(neutron_density, 1, 3)
fig.append_trace(sonic_log, 1, 4)
fig.append_trace(facies_estimation, 1, 5)

fig['layout'].update(height=1000, width=950)
fig['layout'].update( title='Facies QC Curves')

iplot(fig, filename='basic-line')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]  [ (1,4) x4,y4 ]  [ (1,5) x5,y5 ]

